### Homework 4
#### Integer Programming: traveling salesman problem

In [6]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Read in distance matrix.

In [7]:
D = pd.read_csv('five_d.csv', index_col=None, header=None).values
n = len(D)
cities = range(n)
print(D)

[[0 3 4 2 7]
 [3 0 4 6 3]
 [4 4 0 5 8]
 [2 6 5 0 6]
 [7 3 8 6 0]]


Initialize Gurobi model

In [8]:
m = gp.Model('TSP')

Add variables x

In [17]:
#xij, i = n, j = n ,N*N matrics, same size as D. 
x = m.addVars(cities,cities,vtype = GRB.BINARY)

Set objective function to minimize the total tour length

In [19]:
m.setObjective(sum(sum(D[i,j] * x[i,j] for i in cities) for j in cities), GRB.MINIMIZE)

Add constraints

In [30]:
# Constraints here....
#1. xij = xji 
for i in cities:
    for j in cities:
        m.addConstr(x[i,j] == x[j,i])

#2. 
for i in cities:
    for j in cities:
        if i ==j:
            m.addConstr(x[i,j] == 0)
            

#3. sum xij == 2

for i in cities:
    m.addConstr(sum(x[i,j] for j in cities) == 2)



# Helper function for the subtour elimination constraint
def get_power_set(s):
    power_set=[[]]
    for elem in s:
        for sub_set in power_set:
            power_set=power_set+[list(sub_set)+[elem]]
    return power_set

power_set = get_power_set(cities)

# Sub-tour elimination constraint
for S in power_set:
    arcs = gp.LinExpr()
    if len(S) > 0 and len(S) < n:
        for i in S:
            for j in S:
                arcs += x[i,j]
        m.addConstr(arcs <= 2 * (len(S) - 1))

Optimize model

In [31]:
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 240 rows, 25 columns and 890 nonzeros
Model fingerprint: 0x30ecb6db
Variable types: 0 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 6e+00]
Presolve removed 220 rows and 15 columns
Presolve time: 0.02s
Presolved: 20 rows, 10 columns, 80 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)
Found heuristic solution: objective 54.0000000
Found heuristic solution: objective 38.0000000

Root relaxation: cutoff, 4 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0        38.00000   38.00000  0.00%     -    0s

Explored 0 nodes (4 simplex iter

In [35]:
# Print solution
print("Objective value, total distance = ", m.objVal)
print('Since the distance count twice, the actual distance should be',m.objVal/2)

Objective value, total distance =  38.0
Since the distance count twice, the actual distance should be 19.0


In [40]:
route = np.zeros([5,5])
for i in cities:
    for j in cities:
        route[i,j] = x[i,j].x
print(route)

[[ 0. -0.  1.  1. -0.]
 [ 0.  0.  1. -0.  1.]
 [ 1.  1.  0. -0. -0.]
 [ 1.  0.  0.  0.  1.]
 [ 0.  1.  0.  1.  0.]]
